# 42186 Model-based machine learning
- Matteo Piccagnoni s232713
- Gabriel Lanaro s233541
- Manuel Lovo s...

# Topic-aware SPAM message classification in Bayesian setup

## 1. Introduction


Spam messages are annoying, and sometimes dangerous. Classifying them correctly is important, but we often ignore one key aspect: how confident are we in the predictions? 

In this project, we take a more thoughtful approach to spam detection by combining two powerful tools: **topic modeling** and **Bayesian inference**. 

First, we use **Latent Dirichlet Allocation (LDA)** to discover the hidden topics inside SMS messages, this gives us a better understanding of what the messages are about. 
Then, instead of using a standard classifier, we go full **Bayesian** with a **logistic regression model** that doesn’t just make a prediction, it tells us how uncertain that prediction is. 

Everything is built using Pyro, a probabilistic programming library, which makes it easy to define the model and run inference using both SVI and MCMC. 

This notebook walks through the whole process step-by-step: from data cleaning to topic discovery to classification and uncertainty analysis. By the end, we’ll not only have a working spam filter: we will have one that knows when it’s unsure.

In [26]:
import pandas as pd
import numpy as np
import re
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import torch
import pickle

## 2. Dataset and Preprocessing

### 2.1 Dataset

In this project, the **SMS Spam Collection Dataset** has been used. It is a publicly available corpus hosted by the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/228/sms+spam+collection) which contains 5,574 English SMS messages, each labeled as either "ham" (legitimate) or "spam" (unwanted/unsolicited).

The messages were collected from a variety of sources:
Legitimate (ham) messages were gathered from public forums, SMS chat services, and volunteer contributors.
Spam messages were obtained from known spam databases and online archives of promotional SMS campaigns.

Each row in the dataset consists of two fields:
label: A string indicating whether the message is "ham" or "spam".
message: The actual content of the SMS text, written in natural language (English).

A few example rows:

| label | message |
|-------|---------|
| ham   | Are you coming to the party later? |
| spam  | You’ve won a £1000 cash prize! Text WIN to 80086 to claim now. |


The dataset is realistic and includes a broad range of message types, from casual conversations full of slang and abbreviations to marketing promos and scams that mimic legitimate offers. This makes it ideal for studying both semantic patterns (via topic modeling) and predictive classification (spam vs. ham). The class distribution is slightly imbalanced, with around 13% spam and 87% ham, which reflects real-world conditions.

Overall, this dataset offers a compact but rich playground for experimenting with natural language processing, especially when modeling uncertainty and interpretability, as we do in this Bayesian setup.

In [18]:
df_sms = pd.read_csv(
    "SMSSpamCollection", sep="\t", header=None, names=["label", "message"]
)
df_sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [19]:
df = df_sms.copy()

**NEED TO ADD SOME VISUALS TO MAKE IT COOLER AND TO HAVE A BETTER OVERVIEW OF THE DATASET from preprocessing.ipynb**

### 2.2 Preprocessing

Before we can use the text messages in our models, we need to clean and prepare them. Raw SMS messages are often messy including typos, slang, special characters, and unnecessary words that can confuse a model. 
In this section, we’ll go through standard preprocessing steps like **lowercasing**, **removing stopwords**, **tokenizing**, and **stemming**. These transformations help reduce noise and bring the text into a more consistent format, which is especially important for tasks like topic modeling and classification. 

We’ll explain each step as we apply it in the code.

The following setup downloads the list of common English stopwords (like "the", "is", "and") from the NLTK library and initializes the stopword set and a Porter stemmer. These components will be used later to remove common words that carry little semantic meaning and to reduce words to their base form, respectively.

In [20]:
nltk.download("stopwords")
stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/matteopiccagnoni/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


The following function performs standard text preprocessing to prepare SMS messages for further analysis. It includes lowercasing, removal of URLs, numbers, and punctuation, followed by tokenization, stopword removal, and stemming. 
The result is a list of cleaned and normalized tokens for each message.

In [21]:
# Text Cleaning Function
def clean_message(text):
    text = text.lower()
    text = re.sub(r"http\S+|www.\S+", "", text)  # Remove URLs
    text = re.sub(r"\d+", "", text)  # Remove numbers
    text = text.translate(
        str.maketrans("", "", string.punctuation)
    )  # Remove punctuation
    tokens = text.split()
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [stemmer.stem(word) for word in tokens]
    return tokens

The cleaned tokenization function is applied to each message in the dataset using the apply method. A new column, "tokens", is created to store the resulting list of preprocessed tokens for each SMS message.

In [22]:
df["tokens"] = df["message"].apply(clean_message)
df[["message", "tokens"]].head()

,message,tokens
0,"Go until jurong point, crazy.. Available only ...","[go, jurong, point, crazi, avail, bugi, n, gre..."
1,Ok lar... Joking wif u oni...,"[ok, lar, joke, wif, u, oni]"
2,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entri, wkli, comp, win, fa, cup, final,..."
3,U dun say so early hor... U c already then say...,"[u, dun, say, earli, hor, u, c, alreadi, say]"
4,"Nah I don't think he goes to usf, he lives aro...","[nah, dont, think, goe, usf, live, around, tho..."


Above it is provides a quick preview of the structure and content of the DataFrame. At this stage, the output includes the raw message text, and the newly added tokens column, which contains the list of cleaned and preprocessed words extracted from each message. From the raw message to the tokenized version, the following elements were removed: URLs, numbers, punctuation, common stopwords, and words were reduced to their stemmed forms.

>Please notice that in the tokenized output, some words appear in their stemmed form (for example, "crazy" becomes "crazi"). This is a result of the Porter stemming algorithm, which reduces words to their morphological root to group similar terms together. While the resulting stems are not always real words, they help the model treat related terms (e.g., "crazy", "craziness") as the same feature.
It is also worth noting that abbreviations and slang (e.g., "u", "ur", "msg") were intentionally left unchanged. Although these do not follow standard grammar, they often carry important contextual or signals in SMS communication. Removing or expanding them could potentially obscure patterns that distinguish spam from ham in this domain.

In this step, the list of tokens for each message is joined back into a single string to prepare the input for CountVectorizer, which requires text input in string format. The CountVectorizer transforms the preprocessed messages into a bag-of-words (BoW) matrix, where each row represents a message and each column corresponds to a word in the vocabulary, with entries indicating the word count per message.
This BoW representation is a standard format for text modeling and is especially useful for Latent Dirichlet Allocation (LDA), which operates on document-word frequency data. The resulting matrix is converted into a PyTorch tensor to be compatible with Pyro, which is used for building and training the probabilistic topic model.

In [24]:
texts_str = df["tokens"].apply(lambda tokens: " ".join(tokens)) # Join tokens back to strings (for CountVectorizer)

# Create bag-of-words matrix
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts_str)
X_tensor = torch.tensor(X.toarray(), dtype=torch.float) # Convert to torch tensor

print("Input for LDA:", X_tensor.shape)  # should be (num_docs, vocab_size)

Input for LDA: torch.Size([5572, 7099])


The final print statement above shows the shape of the tensor, confirming that the data is now structured as (number of documents, vocabulary size), the expected input format for Pyro’s LDA model.

The example below illustrates the structure of the calculated data, providing a clearer understanding and overview of its format and content.

In [25]:
# Print first cleaned message as tokens
print("\nSample tokens:", df["tokens"].iloc[0])

# Print the BoW vector for the first message (as vector index → count)
print("Sample BoW row:", X_tensor[0].nonzero(as_tuple=True)[0].tolist())
print("Word counts:", X_tensor[0][X_tensor[0] > 0].tolist())

# Print the actual words from the vectorizer
vocab = vectorizer.get_feature_names_out()
print("Words in BoW row:", [vocab[i] for i in X_tensor[0].nonzero(as_tuple=True)[0]])


Sample tokens: ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']
Sample BoW row: [212, 404, 790, 792, 1063, 1280, 2363, 2416, 2453, 3126, 3265, 4621, 6689, 6899]
Word counts: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
Words in BoW row: ['amor', 'avail', 'buffet', 'bugi', 'cine', 'crazi', 'go', 'got', 'great', 'jurong', 'la', 'point', 'wat', 'world']


This code saves the DataFrame with tokens, the fitted vectorizer, and the bag-of-words matrix. These components will be reused in the LDA model to ensure consistency and reproducibility in the topic modeling process.

In [ ]:
# save the DataFrame with tokens
df.to_csv("lda_df.csv", index=False)

# save the vectorizer
with open("lda_vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

# save bow matrix as numpy array
X_array = X.toarray().astype(np.float32)
np.savez_compressed("BoW_X_Array.npz", X_array)

print("File salvati:")
print("- lda_df.csv")
print("- lda_vectorizer.pkl")
print("- BoW_X_Array.npz")

## 3. LDA

## 4. Bayesian Logistic Regression Classifier

## 5. Conclusions 